In [1]:
# takes 12m for 6m->3m result files.
# Read acbl-details.sqlite file and create hand records df.
# warning: Be sure no other apps or windows are open to maximize available memory.
# df columns: board, Hands, DDmakes, Par, (optional: board_record_string, Dealer, Vul, HCP, LoTT)
# Create ../acbl/acbl_hand_records.pkl in a standardized format.

# next steps:
# acbl_hand_records_cleanup.ipynb creates acbl_hand_records_cleaned.pkl from acbl_hand_records.pkl.
# merge-hand-records.ipynb merges acbl and tcg standardized hand records.

# previous steps:
# acbl-json-to-sql.ipynb creates sqlite:///acbl-details.sqlite 

# Requirements
# sqlalchemy

In [2]:
import pandas as pd
import pathlib
import sqlalchemy
import re
from IPython.display import display # needed to define display() method in vscode
import sys

In [ ]:
sys.path.append(str(pathlib.Path.cwd().parent))
sys.path
from mlBridgeLib import mlBridgeLib

In [3]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [4]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')

In [5]:
# using pathlib to create sqlite path.
db_connection_string = 'sqlite:///'+acblPath.joinpath('acbl-details.sqlite').as_posix()
db_connection_string

'sqlite:///e:/bridge/data/acbl/acbl-details.sqlite'

In [6]:
def db_to_frames_dict(engine):
    meta = sqlalchemy.MetaData()
    meta.reflect(engine)
    tables = meta.tables.keys()
    cnx = engine.raw_connection()
    return {t: pd.read_sql(f'SELECT * FROM {t}', cnx) for t in tables}

In [7]:
# takes 2m30s
engine = sqlalchemy.create_engine(db_connection_string) #, echo=True)
with engine.connect() as con:
    if False:
        dfs = db_to_frames_dict(engine)
    else:
        dfs = {}
        #con = engine.raw_connection
        # caution: line might end with LIMIT n or WHERE board_result_id < n
        dfs['events'] = pd.read_sql('SELECT id, club_session FROM events', con)
        dfs['sessions'] = pd.read_sql('SELECT event_id, game_date, hand_record_id FROM sessions', con)
        dfs['hand_records'] = pd.read_sql('SELECT * FROM hand_records', con)
engine.dispose()

In [8]:
for k,df in dfs.items():
    display(k,len(df),df.head())

'events'

375232

,id,club_session
0,100961,Monday Evening
1,110886,Monday Evening
2,114736,Monday Evening
3,126385,Monday Evening
4,126418,Monday Evening


'sessions'

375235

,event_id,game_date,hand_record_id
0,100961,2020-02-17 00:00:00,77908
1,110886,2020-02-24 00:00:00,SHUFFLE
2,114736,2020-03-02 00:00:00,86634
3,126385,2020-03-09 00:00:00,94064
4,126418,2020-03-16 00:00:00,94082


'hand_records'

6540434

,id,created_at,updated_at,board,north_spades,north_hearts,north_diamonds,north_clubs,east_spades,east_hearts,east_diamonds,east_clubs,south_spades,south_hearts,south_diamonds,south_clubs,west_spades,west_hearts,west_diamonds,west_clubs,board_record_hash,board_record_string,hand_record_set_id,dealer,vulnerability,double_dummy_ew,double_dummy_ns,par,auction,comment,notes,points
0,2620401,2020-02-18T06:56:08.000000Z,None,9,10 6,K 3 2,J 2,A J 9 8 7 4,A 8 5 3,Q,K Q 9 6 3,K 5 2,K 7,A J 10 9 8 4,A 5,10 6 3,Q J 9 4 2,7 6 5,10 8 7 4,Q,4f0b32d479c1b503d5a74050b229ed6cde3e5bf7f552c63563fa1f7d4ff73128,S106HK32DJ2CAJ9874SQJ942H765D10874CQSA853HQDKQ963CK52SK7HAJ10984DA5C1063,77908,N,E-W,EW: 3D 3S C2 H3 NT2,NS: 4C 4H 2NT D4 S4,Par: 100 4S*-EW-1,,,,[2620401]
1,2620412,2020-02-18T06:56:08.000000Z,None,20,10 8 5,K J,J 4 3,K J 8 7 3,A J 6 4,10 9 8,10 7 5,A 5 2,K 9 7,Q 7 5 2,A Q 8 2,Q 6,Q 3 2,A 6 4 3,K 9 6,10 9 4,55b30b0a9a5c51f6a5fc04f08b39975963ff4da9428c9aed3f055c7a682f1f6c,S1085HKJDJ43CKJ873SQ32HA643DK96C1094SAJ64H1098D1075CA52SK97HQ752DAQ82CQ6,77908,W,Both,EW: C4 D5 H6 S6 NT6,NS: 2C 2D 1H 1S 1NT,Par: 90 1D-NS+1/1C-NS+1/1NT-NS,,,,[2620412]
2,2620411,2020-02-18T06:56:08.000000Z,None,19,10,7 6 4,A Q 9 2,Q J 6 5 3,K 9 8 4,5,10 8 7 4 3,9 8 7,A 5 2,A Q 9 8 3 2,K J,4 2,Q J 7 6 3,K J 10,6 5,A K 10,dc334d8940b71d553f3f0e2bfee447a2f3e34ae6e6e9cefbbbca81e73f49891b,S10H764DAQ92CQJ653SQJ763HKJ10D65CAK10SK984H5D108743C987SA52HAQ9832DKJC42,77908,S,E-W,EW: 2S C4 D6 H3 NT6,NS: 3C 1D 3H S4/5 NT6,Par: 100 3S*-EW-1,,,,[2620411]
3,2620420,2020-02-18T06:56:08.000000Z,None,28,10,Q J 9 6 3 2,J 9,J 9 3 2,J 7 6,A 10,A K 3,A K Q 8 4,A 9 8 5 4 3 2,K 7 5 4,7,5,K Q,8,Q 10 8 6 5 4 2,10 7 6,5af28684b5610708428de964f17bdc30237797cbdf7233aed26771d974051283,S10HQJ9632DJ9CJ932SKQH8DQ1086542C1076SJ76HA10DAK3CAKQ84SA985432HK754D7C5,77908,W,N-S,EW: 5C 5D 5NT H3 S5,NS: 4H 2S C1/0 D0 NT1/0,Par: -300 6H*-NS-2,,,,[2620420]
4,2620399,2020-02-18T06:56:08.000000Z,None,7,2,Q 10 7 6 2,9 2,A K 10 9 2,Q 10 8 5 4,K 4 3,A 7 6 5,Q,9 6 3,J 9 5,Q 10 8 3,J 7 5,A K J 7,A 8,K J 4,8 6 4 3,d9c0fe840d34c4840ef48b076d75dcc7019ce98d23d474a84dde025c2cf453f1,S2HQ10762D92CAK1092SAKJ7HA8DKJ4C8643SQ10854HK43DA765CQS963HJ95DQ1083CJ75,77908,S,Both,EW: 1C 5D 6S 2NT H6,NS: C6 D2 H6 S1 NT3,Par: -1430 6S-EW,,,,[2620399]


In [9]:
hrs = dfs['hand_records'] # treat hrs as read-only
hrs.head()

,id,created_at,updated_at,board,north_spades,north_hearts,north_diamonds,north_clubs,east_spades,east_hearts,east_diamonds,east_clubs,south_spades,south_hearts,south_diamonds,south_clubs,west_spades,west_hearts,west_diamonds,west_clubs,board_record_hash,board_record_string,hand_record_set_id,dealer,vulnerability,double_dummy_ew,double_dummy_ns,par,auction,comment,notes,points
0,2620401,2020-02-18T06:56:08.000000Z,None,9,10 6,K 3 2,J 2,A J 9 8 7 4,A 8 5 3,Q,K Q 9 6 3,K 5 2,K 7,A J 10 9 8 4,A 5,10 6 3,Q J 9 4 2,7 6 5,10 8 7 4,Q,4f0b32d479c1b503d5a74050b229ed6cde3e5bf7f552c63563fa1f7d4ff73128,S106HK32DJ2CAJ9874SQJ942H765D10874CQSA853HQDKQ963CK52SK7HAJ10984DA5C1063,77908,N,E-W,EW: 3D 3S C2 H3 NT2,NS: 4C 4H 2NT D4 S4,Par: 100 4S*-EW-1,,,,[2620401]
1,2620412,2020-02-18T06:56:08.000000Z,None,20,10 8 5,K J,J 4 3,K J 8 7 3,A J 6 4,10 9 8,10 7 5,A 5 2,K 9 7,Q 7 5 2,A Q 8 2,Q 6,Q 3 2,A 6 4 3,K 9 6,10 9 4,55b30b0a9a5c51f6a5fc04f08b39975963ff4da9428c9aed3f055c7a682f1f6c,S1085HKJDJ43CKJ873SQ32HA643DK96C1094SAJ64H1098D1075CA52SK97HQ752DAQ82CQ6,77908,W,Both,EW: C4 D5 H6 S6 NT6,NS: 2C 2D 1H 1S 1NT,Par: 90 1D-NS+1/1C-NS+1/1NT-NS,,,,[2620412]
2,2620411,2020-02-18T06:56:08.000000Z,None,19,10,7 6 4,A Q 9 2,Q J 6 5 3,K 9 8 4,5,10 8 7 4 3,9 8 7,A 5 2,A Q 9 8 3 2,K J,4 2,Q J 7 6 3,K J 10,6 5,A K 10,dc334d8940b71d553f3f0e2bfee447a2f3e34ae6e6e9cefbbbca81e73f49891b,S10H764DAQ92CQJ653SQJ763HKJ10D65CAK10SK984H5D108743C987SA52HAQ9832DKJC42,77908,S,E-W,EW: 2S C4 D6 H3 NT6,NS: 3C 1D 3H S4/5 NT6,Par: 100 3S*-EW-1,,,,[2620411]
3,2620420,2020-02-18T06:56:08.000000Z,None,28,10,Q J 9 6 3 2,J 9,J 9 3 2,J 7 6,A 10,A K 3,A K Q 8 4,A 9 8 5 4 3 2,K 7 5 4,7,5,K Q,8,Q 10 8 6 5 4 2,10 7 6,5af28684b5610708428de964f17bdc30237797cbdf7233aed26771d974051283,S10HQJ9632DJ9CJ932SKQH8DQ1086542C1076SJ76HA10DAK3CAKQ84SA985432HK754D7C5,77908,W,N-S,EW: 5C 5D 5NT H3 S5,NS: 4H 2S C1/0 D0 NT1/0,Par: -300 6H*-NS-2,,,,[2620420]
4,2620399,2020-02-18T06:56:08.000000Z,None,7,2,Q 10 7 6 2,9 2,A K 10 9 2,Q 10 8 5 4,K 4 3,A 7 6 5,Q,9 6 3,J 9 5,Q 10 8 3,J 7 5,A K J 7,A 8,K J 4,8 6 4 3,d9c0fe840d34c4840ef48b076d75dcc7019ce98d23d474a84dde025c2cf453f1,S2HQ10762D92CAK1092SAKJ7HA8DKJ4C8643SQ10854HK43DA765CQS963HJ95DQ1083CJ75,77908,S,Both,EW: 1C 5D 6S 2NT H6,NS: C6 D2 H6 S1 NT3,Par: -1430 6S-EW,,,,[2620399]


In [10]:
stdhrs = pd.DataFrame()

In [11]:
# takes 1m
#stdhrs['ref'] = hrs.apply(lambda r:tuple(['acbl',r['id'],r['hand_record_set_id']]),axis='columns') # creates column of tuples
stdhrs['ref'] = hrs.apply(lambda r:tuple(['acbl',r['id']]),axis='columns') # creates column of tuples
stdhrs.head()

,ref
0,"(acbl, 2620401)"
1,"(acbl, 2620412)"
2,"(acbl, 2620411)"
3,"(acbl, 2620420)"
4,"(acbl, 2620399)"


In [12]:
hrs.info(), dfs['sessions'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6540434 entries, 0 to 6540433
Data columns (total 32 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   id                   int64 
 1   created_at           object
 2   updated_at           object
 3   board                int64 
 4   north_spades         object
 5   north_hearts         object
 6   north_diamonds       object
 7   north_clubs          object
 8   east_spades          object
 9   east_hearts          object
 10  east_diamonds        object
 11  east_clubs           object
 12  south_spades         object
 13  south_hearts         object
 14  south_diamonds       object
 15  south_clubs          object
 16  west_spades          object
 17  west_hearts          object
 18  west_diamonds        object
 19  west_clubs           object
 20  board_record_hash    object
 21  board_record_string  object
 22  hand_record_set_id   int64 
 23  dealer               object
 24  vulnerability        obj

(None, None)

In [13]:
# takes 5s
stdhrs['hand_record_id'] = hrs['hand_record_set_id'].astype('string')
#sessions = pd.DataFrame()
#sessions['hand_record_id'] = dfs['sessions']['hand_record_id'] #.astype('string')
#sessions['game_date'] = pd.to_datetime(dfs['sessions']['game_date'])
stdhrs = pd.merge(stdhrs,dfs['sessions'][['event_id','game_date','hand_record_id']], how='left', on='hand_record_id')
display(stdhrs.info(),stdhrs.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6540434 entries, 0 to 6540433
Data columns (total 4 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   ref             object
 1   hand_record_id  object
 2   event_id        int64 
 3   game_date       object
dtypes: int64(1), object(3)
memory usage: 249.5+ MB


None

,ref,hand_record_id,event_id,game_date
0,"(acbl, 2620401)",77908,100961,2020-02-17 00:00:00
1,"(acbl, 2620412)",77908,100961,2020-02-17 00:00:00
2,"(acbl, 2620411)",77908,100961,2020-02-17 00:00:00
3,"(acbl, 2620420)",77908,100961,2020-02-17 00:00:00
4,"(acbl, 2620399)",77908,100961,2020-02-17 00:00:00


In [14]:
# obsolete - drop irrelevant columns
#df.drop(['id', 'created_at', 'updated_at', 'north_spades',
#       'north_hearts', 'north_diamonds', 'north_clubs', 'east_spades',
#       'east_hearts', 'east_diamonds', 'east_clubs', 'south_spades',
#       'south_hearts', 'south_diamonds', 'south_clubs', 'west_spades',
#       'west_hearts', 'west_diamonds', 'west_clubs', 'board_record_hash',
#       'auction', 'comment', 'notes', 'points'], axis='columns', inplace=True)
#df

In [15]:
# takes 5s
stdhrs['Board'] = hrs['board'].astype('string').str.zfill(2) # should board be int or zfill(2)?
stdhrs.head()

,ref,hand_record_id,event_id,game_date,Board
0,"(acbl, 2620401)",77908,100961,2020-02-17 00:00:00,09
1,"(acbl, 2620412)",77908,100961,2020-02-17 00:00:00,20
2,"(acbl, 2620411)",77908,100961,2020-02-17 00:00:00,19
3,"(acbl, 2620420)",77908,100961,2020-02-17 00:00:00,28
4,"(acbl, 2620399)",77908,100961,2020-02-17 00:00:00,07


In [16]:
# takes 35s
# todo: implement merging of club_session
club_session = 'Friday Afternoon'.split()[1][0]
stdhrs['EventBoard'] = pd.to_datetime(stdhrs['game_date']).dt.strftime('%y%m%d')+club_session+'_'+stdhrs['Board']
stdhrs.drop('game_date',axis='columns',inplace=True)
stdhrs.head()

,ref,hand_record_id,event_id,Board,EventBoard
0,"(acbl, 2620401)",77908,100961,09,200217A_09
1,"(acbl, 2620412)",77908,100961,20,200217A_20
2,"(acbl, 2620411)",77908,100961,19,200217A_19
3,"(acbl, 2620420)",77908,100961,28,200217A_28
4,"(acbl, 2620399)",77908,100961,07,200217A_07


In [17]:
# rename Dealer and validate
stdhrs['Dealer'] = hrs['dealer']
assert stdhrs['Dealer'].isin(list('NESW')).all()
stdhrs['Dealer'].value_counts()

N    1715179
E    1683771
S    1574755
W    1566729
Name: Dealer, dtype: int64

In [18]:
# takes 2s
# rename vulnerability and validate
stdhrs['Vul'] = hrs['vulnerability']
stdhrs['Vul'].value_counts()

None    1677553
N-S      922626
Both     821180
E-W      805947
EW       779104
NS       775566
All      758458
Name: Vul, dtype: int64

In [19]:
# takes 5s
# rename types of vuls to a common name
vulsd = {'None':'None','All':'Both','NS':'N_S','EW':'E_W','Both':'Both','E-W':'E_W','N-S':'N_S'}
stdhrs['Vul'] = stdhrs['Vul'].replace(vulsd) # example of using dict in series.replace()
stdhrs['Vul'].value_counts()

N_S     1698192
None    1677553
E_W     1585051
Both    1579638
Name: Vul, dtype: int64

In [20]:
## takes 7s
## obsolete - commented out because direction-suits columns are dropped in favor of using board_record_string.
## remake hands into common format
#directions = ['north','east','south','west']
#suits = ['spades','hearts','diamonds','clubs']
#for d,s in zip(directions, suits):
#    ds = d + '_' + s
#    df[ds] = df[ds].str.split(' ').str.join(',')
#df

In [21]:
# takes 10s
stdhrs['board_record_string'] = hrs['board_record_string'].map(lambda x: x.replace('-','').replace('10','T'))
# drop results whose board_record_string contains invalid characters e.g. 2012294
b = ~stdhrs['board_record_string'].str.contains(r'^[SHDCAKQJT98765432]+$')
display(stdhrs[b])
stdhrs.drop(stdhrs[b].index,inplace=True)

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string
2472506,"(acbl, 96495)",3002,3335,14,190604A_14,E,None,SA5?HAQ53DT954CJ2SQ?HK986DK63CK843SKJ97HJTDAQ872CT7ST843H742DJCAQ965


In [22]:
# takes 2m
# remake hands into common format using board_record_string
# timings: map() is slightly faster than apply(). np.vectorize(len) is interesting but 2x slower.
brs = stdhrs['board_record_string']
hands = []
#suit_lengths = []
for s in brs:
    #print(s)
    split_shdc = re.split(r'[SHDC]',s)
    assert len(split_shdc) == 1+4*4
    # create dictionary of NESW each having SHDC cards e.g. for North and Spades: {'N': {'S': 'T7'}}
    #hands.append([{d:{s:h}} for d,s,h in zip(np.repeat(list('NESW'),4),'SHDC'*4,split_shdc[1:])])
    # create a dictionary of tuples (direction, suit) e.g. for North and Spades: {('N', 'S'): 'T7'}
    #hands.append([{(d,s):h} for d,s,h in zip(np.repeat(list('NESW'),4),'SHDC'*4,split_shdc[1:])])
    # create a list of tuples of suits [(),(),(),()]
    hands.append(tuple([tuple(mlBridgeLib.sort_hand(split_shdc[i+1:i+4+1])) for i in [0,8,12,4]])) # needs NWES order!
    #suit_lengths.append(tuple(tuple(len(split_shdc[j+1]) for j in range(i,i+4)) for i in range(0,16,4)))
    # oops, np.vectorize(len) is double speed of for-loop
    #suit_lengths.append(np.vectorize(len)([tuple(split_shdc[i+1:i+4+1]) for i in range(0,16,4)]))
stdhrs['Hands'] = hands
#stdhrs['Suit_Lengths'] = suit_lengths
#stdhrs['Suit_Lengths'] = [tuple(tuple(len(hhh) for hhh in hh) for hh in h) for h in hands]
#stdhrs['Suit_Lengths'] = stdhrs['Hands'].map(lambda h: tuple(tuple(len(hhh) for hhh in hh) for hh in h))
#stdhrs['Suit_Lengths'] = stdhrs['Hands'].apply(lambda h: tuple(np.vectorize(len)(hh) for hh in h))
stdhrs.head()

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))"
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))"
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))"
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))"
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))"


In [23]:
split_shdc

['',
 'AT8',
 'T6',
 'AT987',
 'QJ9',
 'QJ42',
 'J5',
 '5432',
 'AKT',
 '97',
 'Q98742',
 'KQ',
 '532',
 'K653',
 'AK3',
 'J6',
 '8764']

In [24]:
# takes 22s
# display list of reversed hand_records. Count should be 1552 unless new ones are created by ACBL (unlikely).
# alternative fix up is just to delete the rows.
new_bds = [mlBridgeLib.HandToBoardRecordString(h) for h in hands]
stdhrs[stdhrs['board_record_string'] != new_bds]

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands
718594,"(acbl, 783746)",23554,28096,04,191028A_04,W,Both,S56QH238KD48QAC56SHAJT95DK97652CT3S2478TAH6QDTC4JQAS39JKH47D3JC2789K,"((Q65, K832, AQ84, 65), (AT8742, Q6, T, AQJ4), (KJ93, 74, J3, K9872), (, AJT95, K97652, T3))"
718595,"(acbl, 783749)",23554,28096,07,191028A_07,S,Both,S48QH3578D3JC579QSJ6HJ962DT7654CAKS3579AH4TQKD8ACTJS2TKHAD29QKC23468,"((Q84, 8753, J3, Q975), (A9753, KQT4, A8, JT), (KT2, A, KQ92, 86432), (J6, J962, T7654, AK))"
718596,"(acbl, 783752)",23554,28096,10,191028A_10,E,Both,S239JAH67TD257KACST854HKQ4D964CJ63SH2359JDJQC289QKAS67QKH8AD38TC457T,"((AJ932, T76, AK752, ), (, J9532, QJ, AKQ982), (KQ76, A8, T83, T754), (T854, KQ4, 964, J63))"
718597,"(acbl, 783745)",23554,28096,03,191028A_03,S,E_W,S8TJH59TQD3QKC25TSAQ765HAJ3D2CA873S349KH46D4789TC4KS2H278KD56JAC69JQ,"((JT8, QT95, KQ3, T52), (K943, 64, T9874, K4), (2, K872, AJ65, QJ96), (AQ765, AJ3, 2, A873))"
718598,"(acbl, 783748)",23554,28096,06,191028A_06,E,E_W,S46QKAH2TJAD8JC7ASJ72HQ5DA7532CT94S39H67KD69TQC38QKS58TH3489D4KC256J,"((AKQ64, AJT2, J8, A7), (93, K76, QT96, KQ83), (T85, 9843, K4, J652), (J72, Q5, A7532, T94))"
718599,"(acbl, 783751)",23554,28096,09,191028A_09,N,E_W,S49QAHTJD379JKC58S6H84DA64CQJT9763S238KH2356QKAD2CAS57TJH79D58TQC24K,"((AQ94, JT, KJ973, 85), (K832, AKQ6532, 2, A), (JT75, 97, QT85, K42), (6, 84, A64, QJT9763))"
718600,"(acbl, 783744)",23554,28096,02,191028A_02,E,N_S,S245AH4QAD245C59AST6HJ9763DAJ76CJ4S38JKH258TD389TCQS79QHKDQKC23678TK,"((A542, AQ4, 542, A95), (KJ83, T852, T983, Q), (Q97, K, KQ, KT87632), (T6, J9763, AJ76, J4))"
718601,"(acbl, 783747)",23554,28096,05,191028A_05,N,N_S,S28TH34D35KC679JKSQJ53HAK9875DT6C8S79H26TJQD289JC4AS46KAHD47QAC235TQ,"((T82, 43, K53, KJ976), (97, QJT62, J982, A4), (AK64, , AQ74, QT532), (QJ53, AK9875, T6, 8))"
718602,"(acbl, 783743)",23554,28096,01,191028A_01,N,None,S458JH3JAD5TAC24KSQ76H652D64CQJ976S3AH489TQKD389JC8S29TKH7D27QKC35TA,"((J854, AJ3, AT5, K42), (A3, KQT984, J983, 8), (KT92, 7, KQ72, AT53), (Q76, 652, 64, QJ976))"
718603,"(acbl, 783750)",23554,28096,08,191028A_08,W,None,S2KH46TQD256AC56TSQ765HAJ52DQJ87C4S349TJAH89KDC238AS8H37D349TKC79JQK,"((K2, QT64, A652, T65), (AJT943, K98, , A832), (8, 73, KT943, KQJ97), (Q765, AJ52, QJ87, 4))"


In [25]:
stdhrs['board_record_string'] = new_bds
stdhrs.head()

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))"
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))"
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))"
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))"
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))"


In [26]:
# takes 6s
# drop rows which have empty double dummy or par columns
stdhrs['double_dummy_ns'] = hrs['double_dummy_ns'] # temp - will be dropped later
stdhrs['double_dummy_ew'] = hrs['double_dummy_ew'] # temp - will be dropped later
stdhrs['Par'] = hrs['par']
b1 = stdhrs['double_dummy_ns'] != ''
b2 = stdhrs['double_dummy_ew'] != ''
b3 = stdhrs['Par'] != ''
assert all(b1 == b2) and all(b1 == b3)
stdhrs = stdhrs.drop(stdhrs[~b1].index,axis='rows') # dropping empty rows!
stdhrs

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,double_dummy_ns,double_dummy_ew,Par
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))",NS: 4C 4H 2NT D4 S4,EW: 3D 3S C2 H3 NT2,Par: 100 4S*-EW-1
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))",NS: 2C 2D 1H 1S 1NT,EW: C4 D5 H6 S6 NT6,Par: 90 1D-NS+1/1C-NS+1/1NT-NS
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))",NS: 3C 1D 3H S4/5 NT6,EW: 2S C4 D6 H3 NT6,Par: 100 3S*-EW-1
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))",NS: 4H 2S C1/0 D0 NT1/0,EW: 5C 5D 5NT H3 S5,Par: -300 6H*-NS-2
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))",NS: C6 D2 H6 S1 NT3,EW: 1C 5D 6S 2NT H6,Par: -1430 6S-EW
5,"(acbl, 2620396)",77908,100961,04,200217A_04,W,Both,S84HAKJ83D4CQT743SA65HQ62DQ8732CK8SJ972HT5DAKJ96C95SKQT3H974DT5CAJ62,"((84, AKJ83, 4, QT743), (J972, T5, AKJ96, 95), (KQT3, 974, T5, AJ62), (A65, Q62, Q8732, K8))",NS: 4C 4H D5 S6 NT6,EW: 2D C3 H3 S6 NT6,Par: 620 4H-NS
6,"(acbl, 2620410)",77908,100961,18,200217A_18,E,N_S,S872HJ8753DQ873CASAJ65HQ4DAKJ4C874S93HAK6D962CQT963SKQT4HT92DT5CKJ52,"((872, J8753, Q873, A), (93, AK6, 962, QT963), (KQT4, T92, T5, KJ52), (AJ65, Q4, AKJ4, 874))",NS: C4 D4/5 H5 S5 NT5,EW: 3C 2D 1H 1S 2NT,Par: -120 1NT-EW+1
7,"(acbl, 2620400)",77908,100961,08,200217A_08,W,None,S953H98642DJ9CKJ5SAJT6HAJ3DT2CAT96S742HKQ7DK764CQ43SKQ8HT5DAQ853C872,"((953, 98642, J9, KJ5), (742, KQ7, K764, Q43), (KQ8, T5, AQ853, 872), (AJT6, AJ3, T2, AT96))",NS: C4 D6 H6 S4 NT5,EW: 2C 1D 1H 3/2S 2NT,Par: -140 1S-E+2
8,"(acbl, 2620397)",77908,100961,05,200217A_05,N,N_S,S98HAKT83D84CAQ82SQJHQJ65DT2CJ7653SAK752H72DAK96CKTST643H94DQJ753C94,"((98, AKT83, 84, AQ82), (AK752, 72, AK96, KT), (T643, 94, QJ753, 94), (QJ, QJ65, T2, J7653))",NS: 1H C5 D6 S4 NT5,EW: 1C 1D 2S 2NT H6,Par: -120 2NT-EW
9,"(acbl, 2620398)",77908,100961,06,200217A_06,E,E_W,SA53HJT8DQ8743CQ9SKQJT87H954DJCA62S94HAQ62DAT965C54S62HK73DK2CKJT873,"((A53, JT8, Q8743, Q9), (94, AQ62, AT965, 54), (62, K73, K2, KJT873), (KQJT87, 954, J, A62))",NS: 1C D6 H5 S4 NT5,EW: 1D 2H 3S C5 NT5,Par: -140 2S-EW+1


In [27]:
# renumber index after having dropped rows!
stdhrs = stdhrs.reset_index(drop=True)
stdhrs

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,double_dummy_ns,double_dummy_ew,Par
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))",NS: 4C 4H 2NT D4 S4,EW: 3D 3S C2 H3 NT2,Par: 100 4S*-EW-1
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))",NS: 2C 2D 1H 1S 1NT,EW: C4 D5 H6 S6 NT6,Par: 90 1D-NS+1/1C-NS+1/1NT-NS
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))",NS: 3C 1D 3H S4/5 NT6,EW: 2S C4 D6 H3 NT6,Par: 100 3S*-EW-1
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))",NS: 4H 2S C1/0 D0 NT1/0,EW: 5C 5D 5NT H3 S5,Par: -300 6H*-NS-2
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))",NS: C6 D2 H6 S1 NT3,EW: 1C 5D 6S 2NT H6,Par: -1430 6S-EW
5,"(acbl, 2620396)",77908,100961,04,200217A_04,W,Both,S84HAKJ83D4CQT743SA65HQ62DQ8732CK8SJ972HT5DAKJ96C95SKQT3H974DT5CAJ62,"((84, AKJ83, 4, QT743), (J972, T5, AKJ96, 95), (KQT3, 974, T5, AJ62), (A65, Q62, Q8732, K8))",NS: 4C 4H D5 S6 NT6,EW: 2D C3 H3 S6 NT6,Par: 620 4H-NS
6,"(acbl, 2620410)",77908,100961,18,200217A_18,E,N_S,S872HJ8753DQ873CASAJ65HQ4DAKJ4C874S93HAK6D962CQT963SKQT4HT92DT5CKJ52,"((872, J8753, Q873, A), (93, AK6, 962, QT963), (KQT4, T92, T5, KJ52), (AJ65, Q4, AKJ4, 874))",NS: C4 D4/5 H5 S5 NT5,EW: 3C 2D 1H 1S 2NT,Par: -120 1NT-EW+1
7,"(acbl, 2620400)",77908,100961,08,200217A_08,W,None,S953H98642DJ9CKJ5SAJT6HAJ3DT2CAT96S742HKQ7DK764CQ43SKQ8HT5DAQ853C872,"((953, 98642, J9, KJ5), (742, KQ7, K764, Q43), (KQ8, T5, AQ853, 872), (AJT6, AJ3, T2, AT96))",NS: C4 D6 H6 S4 NT5,EW: 2C 1D 1H 3/2S 2NT,Par: -140 1S-E+2
8,"(acbl, 2620397)",77908,100961,05,200217A_05,N,N_S,S98HAKT83D84CAQ82SQJHQJ65DT2CJ7653SAK752H72DAK96CKTST643H94DQJ753C94,"((98, AKT83, 84, AQ82), (AK752, 72, AK96, KT), (T643, 94, QJ753, 94), (QJ, QJ65, T2, J7653))",NS: 1H C5 D6 S4 NT5,EW: 1C 1D 2S 2NT H6,Par: -120 2NT-EW
9,"(acbl, 2620398)",77908,100961,06,200217A_06,E,E_W,SA53HJT8DQ8743CQ9SKQJT87H954DJCA62S94HAQ62DAT965C54S62HK73DK2CKJT873,"((A53, JT8, Q8743, Q9), (94, AQ62, AT965, 54), (62, K73, K2, KJT873), (KQJT87, 954, J, A62))",NS: 1C D6 H5 S4 NT5,EW: 1D 2H 3S C5 NT5,Par: -140 2S-EW+1


In [28]:
# takes 35s
# clean double dummy columns. rewrite into standarized form.
# todo: cleaning and validations need to be done in previous notebook
ddd = ['double_dummy_ns','double_dummy_ew']
for d in ddd:
    assert stdhrs[d].str.startswith(d[-2:].upper()+':').all()
    stdhrs[d] =  stdhrs[d].str.replace('NT','N').replace(r'\s+',' ',regex=True) # remove extra spaces
    assert (~stdhrs[d].str.contains('NT')).all()
    b = (stdhrs[d].str.count(' ') != 5) | ~stdhrs[d].str.startswith(d[-2:].upper()+': ')
    wrong_counts = stdhrs[d][b]
    display(len(wrong_counts),wrong_counts) # about 5000 rejects
    stdhrs.drop(wrong_counts.index,inplace=True)

3835

16001             NS: C3/4 3H -/1N N6/7 S4 2/3D
114387            NS: C6 4H 4/3N 1/-S S7/6 5/4D
114395       NS: -/1C C6/7 2H -/1N N6/7 3S 1/2D
114403            NS: 2/3C 3/4H 2N 5S -/1D D6/7
114413              NS: C5 4H 2/-N N8/6 3S 4/3D
114415          NS: C4/5 H4 N3/4 -/1S S6/7 D2/3
130954         NS: 1S 2C 1/-H H7/6 2N 1/-D D7/6
130988                NS: D6 -/1S S6/7 N6 C6 1H
130994                NS: 2D S5 -/2N N6/8 C5 1H
131032              NS: 1/-H H7/6 1N S5/4 C5 3D
131352     NS: 1/-N N7/6 S4 4/3D 1/-H H7/6 3/2C
131383                NS: S3 N5 H3 D4 1/-C C7/6
131536                NS: 2N D3 4S 4C 1/-H H7/6
131544                NS: -/1N N6/7 1D S4 C6 H6
131547         NS: 1N 1D 1/-S S7/6 1/-C C7/6 2H
131565                NS: -/1C C6/7 D2 N6 2H S3
131579         NS: 1N 3D -/1S S6/7 1H -/1C C6/7
131583              NS: 1/-N N7/6 3D 2S 2H C5/4
131608                NS: 4S D6 1N 1/-H H7/6 5C
131613        NS: S4/5 1/2D -/1N N6/7 H5/6 C5/6
131629              NS: 1S 1D -/1N N6/7 

2182

16000                EW: -/1C C6/7 H5/6 N5/6 4S D6
95543                  EW: 1/-H H7/6 6/5N 6D 1S 4C
114389                 EW: -/1C C6/7 3H 2N S5/6 2D
114412                   EW: 1/-C C7/6 H5 1N S6 2D
130962                   EW: 2S C5 H6 1/-N N7/6 1D
130983                   EW: D6 3S 3N 4C 1/-H H7/6
130985                   EW: D5 2S -/1N N6/7 2C 2H
131010                   EW: H5 N5 1S C6 1/-D D7/6
131017                   EW: 2H 1/-N N7/6 S5 C4 2D
131020                   EW: -/1H H6/7 4N 3S 3C 4D
131346                   EW: N6 S4 2D 1H -/1C C5/7
131378                   EW: -/2N N6/8 S5 2D 2H C6
131391     EW: S4 -/1N N6/7 -/1H H6/7 D5 -/1C C6/7
131549                   EW: C4 1/-D D7/6 N5 2H S3
131573               EW: C3 D5/6 N5/4 H5 1/-S S7/6
131590                   EW: N6 D6 1/-S S7/6 H6 3C
131612                   EW: -/1S S6/7 D5 N5 H6 1C
131646                 EW: 1/2H D6 1S -/1N N6/7 3C
131652                 EW: 2H -/1D D6/7 4S 1N C5/6
131683          EW: 1/-N N7/6 4

In [29]:
# takes 2m
# rewrite double dummy columns into more usable format
makes = []
zdd = zip(stdhrs['double_dummy_ns'],stdhrs['double_dummy_ew'])
#display(stdhrs.head(1))
for nsewdd in zdd:
    #print(nsewdd)
    #l = []
    #makes.append(l)
    tricks = {}
    for dd in nsewdd:
        split_space = dd.split(' ')
        #print(split_space)
        ds = split_space[0][:2].upper()
        #print(ds)
        assert len(split_space) == 6
        for ddsuit in split_space[1:]:
            #print(ddsuit)
            found = re.findall(r'(^([CDHSN])(\d+)$)|(^(\d+)([CDHSN])$)|(^([CDHSN])(\d+)\/(\d+)$)|(^(\d+)\/(\d+)([CDHSN])$)',ddsuit)
            assert len(found) == 1
            r = found[0]
            #print(r)
            if r[0] != '': # S4 -- direction makes 4 tricks in spades
                assert r[1] in 'CDHSN' and r[2].isdigit()
                suit = r[1]
                levelNE = levelSW = int(r[2])
            elif r[3] != '': # 4S -- direction makes 10 tricks in spades
                assert r[4].isdigit() and r[5] in 'CDHSN'
                levelNE = levelSW = int(r[4])+6
                suit = r[5]
            elif r[6] != '': # S4/5 -- direction (N or E) makes 4 tricks in spades, direction (S or W) makes 5 tricks
                assert r[7] in 'CDHSN' and r[8].isdigit() and r[9].isdigit()
                suit = r[7]
                levelNE = int(r[8])
                levelSW = int(r[9])
            elif r[10] != '': # S4/5 -- direction (N or E) makes 10 tricks in spades, direction (S or W) makes 11 tricks
                assert r[11].isdigit() and r[12].isdigit() and r[13] in 'CDHSN'
                levelNE = int(r[11])+6
                levelSW = int(r[12])+6
                suit = r[13]
            else:
                assert False
            assert ds[0]+suit not in tricks
            tricks[ds[0]+suit] = levelNE
            assert ds[1]+suit not in tricks
            tricks[ds[1]+suit] = levelSW
    #print(tricks)
    assert len(tricks) == 4*5
    t = tuple(tuple(tricks[d+s] for s in 'CDHSN') for d in 'NESW') # controls order of tuple # use dict instead of tuple?
    makes.append(t)
stdhrs['DDmakes'] = makes

In [30]:
stdhrs['DDmakes']

0                ((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))
1                    ((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))
2                    ((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))
3            ((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))
4                ((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))
5                ((10, 5, 10, 6, 6), (3, 8, 3, 6, 6), (10, 5, 10, 6, 6), (3, 8, 3, 6, 6))
6                    ((4, 4, 5, 5, 5), (9, 8, 7, 7, 8), (4, 5, 5, 5, 5), (9, 8, 7, 7, 8))
7                    ((4, 6, 6, 4, 5), (8, 7, 7, 9, 8), (4, 6, 6, 4, 5), (8, 7, 7, 8, 8))
8                    ((5, 6, 7, 4, 5), (7, 7, 6, 8, 8), (5, 6, 7, 4, 5), (7, 7, 6, 8, 8))
9                    ((7, 6, 5, 4, 5), (5, 7, 8, 9, 5), (7, 6, 5, 4, 5), (5, 7, 8, 9, 5))
10                   ((9, 7, 6, 9, 7), (3, 6, 6, 4, 5), (9, 7, 6, 9, 7), (3, 6, 6, 4, 5))
11        

In [31]:
# no need for double_dummy_?? anymore
stdhrs.drop(['double_dummy_ew','double_dummy_ns'],axis='columns',inplace=True)

In [32]:
# takes 25s
# Create Par score column in standardized format
# rename par column. rewrite as list 
# too complex for one regex, if at all possible, so must iterate and split().
# todo: eliminate for-loop by using replace() with a list of regex? Or using map/apply?
assert stdhrs['Par'].str.startswith('Par: ').all()
parl = []
for v in stdhrs['Par']:
    #print(v)
    split_comma = v.split(' ')
    assert split_comma[0] == 'Par:'
    assert len(split_comma) == 3
    score = int(split_comma[1])
    split_slash = split_comma[2].replace('NT','N').split('/')
    assert len(split_slash) > 0
    pars = []
    parl.append((score, pars))
    if score == 0: # all pass is par score
        pars.append((0,'','','',0))
        continue
    for contract in split_slash:
        bid = re.match('(\d)([CDHSN])(\**)-(NS|EW|[NSEW])([\+\-]\d)?',contract)
        assert len(bid.groups()) > 0
        #print(bid.groups())
        level, suit, double, direction, result = bid.groups()
        pars.append((int(level),suit,double,direction,0 if result is None else int(result)))
#display(parl)
stdhrs['Par'] = parl
stdhrs.head(100)

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(100, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))"
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))"
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(100, [(3, S, *, EW, -1)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))"
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-300, [(6, H, *, NS, -2)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))"
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))","(-1430, [(6, S, , EW, 0)])","((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))"
5,"(acbl, 2620396)",77908,100961,04,200217A_04,W,Both,S84HAKJ83D4CQT743SA65HQ62DQ8732CK8SJ972HT5DAKJ96C95SKQT3H974DT5CAJ62,"((84, AKJ83, 4, QT743), (J972, T5, AKJ96, 95), (KQT3, 974, T5, AJ62), (A65, Q62, Q8732, K8))","(620, [(4, H, , NS, 0)])","((10, 5, 10, 6, 6), (3, 8, 3, 6, 6), (10, 5, 10, 6, 6), (3, 8, 3, 6, 6))"
6,"(acbl, 2620410)",77908,100961,18,200217A_18,E,N_S,S872HJ8753DQ873CASAJ65HQ4DAKJ4C874S93HAK6D962CQT963SKQT4HT92DT5CKJ52,"((872, J8753, Q873, A), (93, AK6, 962, QT963), (KQT4, T92, T5, KJ52), (AJ65, Q4, AKJ4, 874))","(-120, [(1, N, , EW, 1)])","((4, 4, 5, 5, 5), (9, 8, 7, 7, 8), (4, 5, 5, 5, 5), (9, 8, 7, 7, 8))"
7,"(acbl, 2620400)",77908,100961,08,200217A_08,W,None,S953H98642DJ9CKJ5SAJT6HAJ3DT2CAT96S742HKQ7DK764CQ43SKQ8HT5DAQ853C872,"((953, 98642, J9, KJ5), (742, KQ7, K764, Q43), (KQ8, T5, AQ853, 872), (AJT6, AJ3, T2, AT96))","(-140, [(1, S, , E, 2)])","((4, 6, 6, 4, 5), (8, 7, 7, 9, 8), (4, 6, 6, 4, 5), (8, 7, 7, 8, 8))"
8,"(acbl, 2620397)",77908,100961,05,200217A_05,N,N_S,S98HAKT83D84CAQ82SQJHQJ65DT2CJ7653SAK752H72DAK96CKTST643H94DQJ753C94,"((98, AKT83, 84, AQ82), (AK752, 72, AK96, KT), (T643, 94, QJ753, 94), (QJ, QJ65, T2, J7653))","(-120, [(2, N, , EW, 0)])","((5, 6, 7, 4, 5), (7, 7, 6, 8, 8), (5, 6, 7, 4, 5), (7, 7, 6, 8, 8))"
9,"(acbl, 2620398)",77908,100961,06,200217A_06,E,E_W,SA53HJT8DQ8743CQ9SKQJT87H954DJCA62S94HAQ62DAT965C54S62HK73DK2CKJT873,"((A53, JT8, Q8743, Q9), (94, AQ62, AT965, 54), (62, K73, K2, KJT873), (KQJT87, 954, J, A62))","(-140, [(2, S, , EW, 1)])","((7, 6, 5, 4, 5), (5, 7, 8, 9, 5), (7, 6, 5, 4, 5), (5, 7, 8, 9, 5))"


In [33]:
stdhrs[stdhrs['Par'].map(lambda x: x[0]==0)] # pars which are passed out

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
112689,"(acbl, 3055964)",90800,121387,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"
146237,"(acbl, 662960)",19935,23426,32,191019A_32,W,E_W,SA7HAQ63DA765CAK5SKJT83HJ9742D8CQTS62HK8DKQJT4CJ984SQ954HT5D932C7632,"((A7, AQ63, A765, AK5), (62, K8, KQJT4, J984), (Q954, T5, 932, 7632), (KJT83, J9742, 8, QT))","(0, [(0, , , , 0)])","((6, 5, 6, 6, 6), (6, 6, 6, 6, 6), (6, 5, 6, 6, 6), (6, 6, 6, 6, 6))"
166178,"(acbl, 3060444)",90935,121620,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"
219173,"(acbl, 3058440)",90874,121498,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"
295265,"(acbl, 3054828)",90766,121336,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"
315487,"(acbl, 3055312)",90780,121354,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"
322948,"(acbl, 3054211)",90748,121305,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"
345786,"(acbl, 3054864)",90767,121338,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"
382738,"(acbl, 3056469)",90815,121415,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"
432785,"(acbl, 3057196)",90837,121444,21,200309A_21,N,N_S,S8HAKQJDJ9764CKJ3SAQJ9HT7654DT32C8SK63H2DKQ85CA7652ST7542H983DACQT94,"((8, AKQJ, J9764, KJ3), (K63, 2, KQ85, A7652), (T7542, 983, A, QT94), (AQJ9, T7654, T32, 8))","(0, [(0, , , , 0)])","((6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5), (6, 6, 6, 6, 5))"


In [34]:
b = stdhrs['Par'].map(lambda x: x[0])
b.head()

0     100
1      90
2     100
3    -300
4   -1430
Name: Par, dtype: int64

In [35]:
len(stdhrs)

3355150

In [36]:
# takes 35s
acbl_hand_records_filename = acblPath.joinpath('acbl_hand_records.pkl')
stdhrs.to_pickle(acbl_hand_records_filename)

In [37]:
# takes 15s
stdhrs = pd.read_pickle(acbl_hand_records_filename)
display(len(stdhrs),stdhrs)

3355150

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(100, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))"
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))"
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(100, [(3, S, *, EW, -1)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))"
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-300, [(6, H, *, NS, -2)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))"
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))","(-1430, [(6, S, , EW, 0)])","((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))"
5,"(acbl, 2620396)",77908,100961,04,200217A_04,W,Both,S84HAKJ83D4CQT743SA65HQ62DQ8732CK8SJ972HT5DAKJ96C95SKQT3H974DT5CAJ62,"((84, AKJ83, 4, QT743), (J972, T5, AKJ96, 95), (KQT3, 974, T5, AJ62), (A65, Q62, Q8732, K8))","(620, [(4, H, , NS, 0)])","((10, 5, 10, 6, 6), (3, 8, 3, 6, 6), (10, 5, 10, 6, 6), (3, 8, 3, 6, 6))"
6,"(acbl, 2620410)",77908,100961,18,200217A_18,E,N_S,S872HJ8753DQ873CASAJ65HQ4DAKJ4C874S93HAK6D962CQT963SKQT4HT92DT5CKJ52,"((872, J8753, Q873, A), (93, AK6, 962, QT963), (KQT4, T92, T5, KJ52), (AJ65, Q4, AKJ4, 874))","(-120, [(1, N, , EW, 1)])","((4, 4, 5, 5, 5), (9, 8, 7, 7, 8), (4, 5, 5, 5, 5), (9, 8, 7, 7, 8))"
7,"(acbl, 2620400)",77908,100961,08,200217A_08,W,None,S953H98642DJ9CKJ5SAJT6HAJ3DT2CAT96S742HKQ7DK764CQ43SKQ8HT5DAQ853C872,"((953, 98642, J9, KJ5), (742, KQ7, K764, Q43), (KQ8, T5, AQ853, 872), (AJT6, AJ3, T2, AT96))","(-140, [(1, S, , E, 2)])","((4, 6, 6, 4, 5), (8, 7, 7, 9, 8), (4, 6, 6, 4, 5), (8, 7, 7, 8, 8))"
8,"(acbl, 2620397)",77908,100961,05,200217A_05,N,N_S,S98HAKT83D84CAQ82SQJHQJ65DT2CJ7653SAK752H72DAK96CKTST643H94DQJ753C94,"((98, AKT83, 84, AQ82), (AK752, 72, AK96, KT), (T643, 94, QJ753, 94), (QJ, QJ65, T2, J7653))","(-120, [(2, N, , EW, 0)])","((5, 6, 7, 4, 5), (7, 7, 6, 8, 8), (5, 6, 7, 4, 5), (7, 7, 6, 8, 8))"
9,"(acbl, 2620398)",77908,100961,06,200217A_06,E,E_W,SA53HJT8DQ8743CQ9SKQJT87H954DJCA62S94HAQ62DAT965C54S62HK73DK2CKJT873,"((A53, JT8, Q8743, Q9), (94, AQ62, AT965, 54), (62, K73, K2, KJT873), (KQJT87, 954, J, A62))","(-140, [(2, S, , EW, 1)])","((7, 6, 5, 4, 5), (5, 7, 8, 9, 5), (7, 6, 5, 4, 5), (5, 7, 8, 9, 5))"
